In [31]:
import sys
import os
import torch
import evaluate

# Add project root (one level up) to PYTHONPATH
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification,  AutoConfig
from src.preprocess import preprocess_texts


In [32]:
device = 0 if torch.cuda.is_available() else -1

In [42]:
print(torch.cuda.is_available())


False


In [23]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

In [26]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
dataset = load_dataset("tweet_eval", "sentiment")

def preprocess_batch(batch):
    batch["text"] = preprocess_texts(batch["text"])
    return batch

cleaned_ds = dataset.map(preprocess_batch, batched=True)

In [28]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 45615
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 12284
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [39]:
test_ds = cleaned_ds["test"]
texts = list(test_ds["text"])
labels = list(test_ds["label"])

In [40]:
type(texts)

list

In [41]:
sentiment_task = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)
preds = sentiment_task(texts)

Device set to use cpu


KeyboardInterrupt: 